In [24]:
import numpy as np
import pandas as pd
import sqlite3
import datetime

from utils import round_integers, today_date_as_epoch

grajki = [
    "ESTERAD#8174023",
    "NOTEVIDIUS#5532788",
    "BORUCHOMIR#7172961",
    "DAWNY-KECZUP#7192166",
    "GITAN#2046256",
    "WORRIED-BLADE115#1720700"
]

def aggregate_stats(data, player_name, last_hours=24):
    df = data[(data['utcStartSeconds'] > today_date_as_epoch()) & (data['player'] == player_name)]
    # df = data[(data['player'] == player_name)]
    
    result_meta = pd.DataFrame({
        "matches": df["matchID"].nunique(),
        "highestMultikill": df["highestMultikill"].max(),
        "scorePerMinuteAvg": df["scorePerMinute"].mean(),
        "highestStreak": df["highestStreak"].max(),
        "mostKillsInMatch": df['kills'].max(),
        "mostDeathsInMatch": df['deaths'].max(),
        "mostObjectivesInMatch": df['objectives'].max()
    }, index=[0])
    sums = pd.DataFrame(df[['kills', 'deaths', 'score', 'damageDealt', 'objectives', 'headshots', 'assists',
               'multikills', 'shotsFired', 'shotsLanded', 'shotsMissed', 'hits', 'timePlayed',
               'suicides', 'shots', 'timePlayedAlive', 'ekia']].sum()).T
    result = pd.concat([result_meta, sums], axis=1)
    result['kdRatio'] = round(result['kills'] / result['deaths'], 3)
    result['ekiadRatio'] = round(result['ekia'] / result['deaths'], 3)
    result['accuracy'] = round(result['shotsLanded'] / result['shotsFired'], 3)
    result['matchesWon'] = df[df["result"] == "win"].shape[0]
    result = round_integers(result)
    return result[['matches', 'matchesWon', 'kills', 'deaths', 'objectives', 'headshots', 'assists',
                   'highestMultikill', 'multikills', 'highestStreak', 'scorePerMinuteAvg', 'ekia',
                   'kdRatio', 'ekiadRatio', 'score', 'damageDealt',
                   'shotsFired', 'shotsLanded', 'accuracy',
                   'suicides', 'mostKillsInMatch',
                   'mostDeathsInMatch', 'mostObjectivesInMatch']]

In [7]:
# result = pd.DataFrame()

# for player_id in grajki:
#     conn = sqlite3.connect("data/cod_stats.db")
#     player_name = player_id.split("#")[0].lower().replace("-", "_")
#     df = pd.read_sql(
#         sql=f"SELECT * FROM {player_name}",
#         con=conn
#     )
#     df3 = aggregate_stats(df)
#     df3.index = [player_name]
#     result = pd.concat([result, df3])

# for col in result.columns:
#     result[col] = result[col].astype(str)
# result = result.T

In [25]:
result = pd.DataFrame()
conn = sqlite3.connect("data/cod_stats.db")
df = pd.read_sql(
    sql=f"SELECT * FROM stats",
    con=conn
)

for player_id in grajki:
    player_name = player_id.split("#")[0].lower().replace("-", "_")
    df3 = aggregate_stats(df, player_name)
    df3.index = [player_name]
    result = pd.concat([result, df3])

for col in result.columns:
    result[col] = result[col].astype(str)
result = result.T

In [26]:
result

,esterad,notevidius,boruchomir,dawny_keczup,gitan,worried_blade115
matches,50,83,87,41,30,25
matchesWon,30,55,57,28,18,15
kills,909,1296,1355,489,526,214
deaths,903,1442,1521,706,556,305
objectives,458,845,1018,244,282,129
headshots,171,214,220,79,87,46
assists,238,354,383,144,169,64
highestMultikill,3,4,3,3,4,3
multikills,73,99,106,31,57,10
highestStreak,11,12,8,9,9,7


In [8]:
from utils import round_integers

records_cols_dict = {
        "highestScore": "score",
        "highestDamageDealt": "damageDealt",
        "highestKdRatio": "kdRatio",
        "highestEkiadRatio": "ekiadRatio",
        "highestMultikill": "highestMultikill",
        "highestScorePerMinute": "scorePerMinute",
        "highestStreak": "highestStreak",
        "mostKillsInMatch": 'kills',
        "mostDeathsInMatch": 'deaths',
        "mostObjectivesInMatch": 'objectives',
        "mostHeadshotsInMatch": "headshots",
        "mostEkiaInMatch": "ekia"
}


def calc_records(data):
    df = round_integers(data).copy()
    records = pd.DataFrame({
        "highestScore": str(df["score"].max()),
        "highestDamageDealt": str(df["damageDealt"].max()),
        "highestKdRatio": str(df["kdRatio"].max()),
        "highestEkiadRatio": str(df["ekiadRatio"].max()),
        "highestMultikill": str(df["highestMultikill"].max()),
        "highestScorePerMinute": str(df["scorePerMinute"].max()),
        "highestStreak": str(df["highestStreak"].max()),
        "mostKillsInMatch": str(df['kills'].max()),
        "mostEkiaInMatch": str(df['ekia'].max()),
        "mostDeathsInMatch": str(df['deaths'].max()),
        "mostObjectivesInMatch": str(df['objectives'].max()),
        "mostHeadshotsInMatch": str(df["headshots"].max())
    }, index=["record"])
    records = records.T
    records['player'] = ''
    records['date'] = ''
    records['map'] = ''
    records['mode'] = ''
    for record in records.index:
        record_col = records_cols_dict[record]
        for desc_col in ['player', 'date', 'map', 'mode']:
            records[desc_col][record] = df[df[record_col].astype(str) == records.loc[record]['record']][desc_col].values[0]

    today_date = datetime.date.today().strftime("%Y-%m-%d")

    records['new'] = np.where(records['date'] == today_date, '<-- NOWY REKORD', '')
    
    return records

all_data = df.copy()
records_df = calc_records(all_data)

In [9]:
result

,esterad,notevidius,boruchomir,dawny_keczup,gitan,worried_blade115
matches,2,10,10,5,0,10
matchesWon,1,7,7,4,0,7
kills,32,154,146,62,0,80
deaths,39,109,120,76,0,98
objectives,36,108,124,50,0,67
headshots,13,37,15,15,0,16
assists,14,22,29,21,0,26
highestMultikill,2,3,3,3,0,2
multikills,3,8,13,5,0,6
highestStreak,4,12,7,4,0,7


In [10]:
records_df

,record,player,date,map,mode,new
highestScore,9330,esterad,2023-09-13,mp_cliffhanger,dem,
highestDamageDealt,7079,notevidius,2023-09-08,mp_tundra,dom,
highestKdRatio,7.0,notevidius,2023-09-18,mp_slums_rm,ctf,<-- NOWY REKORD
highestEkiadRatio,7.0,notevidius,2023-09-18,mp_slums_rm,ctf,<-- NOWY REKORD
highestMultikill,4,gitan,2023-09-13,mp_cliffhanger,dem,
highestScorePerMinute,899,gitan,2023-09-15,mp_dune,dm,
highestStreak,12,notevidius,2023-09-18,mp_hijacked_rm,control,<-- NOWY REKORD
mostKillsInMatch,43,notevidius,2023-09-08,mp_tundra,dom,
mostEkiaInMatch,51,notevidius,2023-09-08,mp_tundra,dom,
mostDeathsInMatch,47,gitan,2023-09-13,mp_village_rm,dem,


---
## Individual records

In [18]:
def get_individual_records_query(record_col: str, record_name: str) -> str:
    return f"""
    WITH
    records AS (
        SELECT player, MAX({record_col}) AS {record_name}
        FROM stats
        GROUP BY 1
        ORDER BY 2 DESC
    )
    SELECT s.player, s.map, s.mode, s.date, r.{record_name}
    FROM stats s
    JOIN records r
        ON s.player = r.player
        AND s.{record_col} = r.{record_name}
    ORDER BY r.{record_name} DESC, s.date ASC;
    """

In [20]:
conn = sqlite3.connect("data/cod_stats.db")

for record_name in records_cols_dict:
    record_col = records_cols_dict[record_name]

    df = pd.read_sql(
        sql=get_individual_records_query(record_col, record_name),
        con=conn
    )
    df

In [21]:
df

,player,map,mode,date,mostEkiaInMatch
0,notevidius,mp_tundra,dom,2023-09-08,51.0
1,esterad,mp_slums_rm,dom,2023-09-14,44.0
2,gitan,mp_cartel,koth,2023-09-15,44.0
3,boruchomir,mp_zoo_rm,koth,2023-09-12,43.0
4,boruchomir,mp_drivein_rm,dem,2023-09-18,43.0
5,dawny_keczup,mp_paintball_rm,dem,2023-09-17,30.0
6,worried_blade115,mp_nuketown6,tdm,2023-09-15,29.0
